# Experiment records
- Trained model name: resnet_trained_model_digit_with_printed_v1_new_finetune_13_08_22_epoch_5.h5 (Fine-tuned on base model)
- Trained for: 22 epochs
- Augmentations used: No
- Accuracy achieved: Same as experiment performed over NSIT dataset (Input as black and white images)
- Dataset used: https://www.nist.gov/srd/nist-special-database-19
- Description of experiment: To show that irrespective of color (Red, blue, or green) of the input, the output/ prediction of the model changes by approx 1-2%. 

In [1]:
import tensorflow as tf
import sys  
import os
sys.path.append(os.path.realpath('../../src'))
# from predict import pred_using_h5_digit, pred_using_tflite_model
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

2022-12-09 10:09:14.823724: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 10:09:14.823744: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_mask(img):
#     print(img)
    image = cv2.imread(img, cv2.IMREAD_UNCHANGED)    
    #make mask of where the transparent bits are
    trans_mask = image[:,:,3] == 0
    #replace areas of transparency with white and not transparent
    image[trans_mask] = [255, 255, 255, 255]
    #new image without alpha channel...
    new_img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    return new_img

In [3]:
# To change the color of input - To Blue, Green, or Red
def change_color(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_OTSU)
    contours, heirarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    conts = contours[-1]
    cv2.fillPoly(img, pts=[conts], color =(255,0,0))
    return img

In [4]:
model = tf.keras.models.load_model('/home/venkateshiyer/react-native-saral-sdk/ml_models/handwritten_digits/models/pre-trained_model/resnet_trained_model_digit_with_printed_v1_new_finetune_13_08_22_epoch_5.h5')

2022-12-09 10:09:21.198811: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-09 10:09:21.198846: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ti-lap-492): /proc/driver/nvidia/version does not exist
2022-12-09 10:09:21.199127: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def pred_using_h5_digit(model, path, gt = None, prediction= None, wrong_imgs=None):
    result = {}
    gt=[]
    prediction = []
    wrong_imgs = {}
    wrong_count=0
    for img1 in sorted(glob.iglob(path)):
        img= change_color(img1)
        img=cv2.resize(img,(28,28))
        img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.astype('float32') / 255.
        img= np.reshape(img,(1,28,28,1))
        res=model.predict(img)
        pred=res[0].argmax(axis=0)
        ground_truth = int(img1.split('/')[-2])
        gt.append(int(ground_truth))
        prediction.append(pred)
        result[img1] = pred
        if pred!= int(ground_truth):
            wrong_count+=1
            wrong_imgs[img1] = pred
    accuracy = (len(glob.glob(path))-wrong_count)/len(glob.glob(path))
    return result, accuracy, gt, prediction, wrong_imgs

In [13]:
# For 1

In [14]:
path = '/home/venkateshiyer/handwritten_digits_test/1/*'
result, accuracy, gt, prediction, wrong_imgs = pred_using_h5_digit(model,path)

2022-12-08 16:33:43.123008: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [15]:
print(len(wrong_imgs))
print(wrong_imgs)

2
{'/home/venkateshiyer/handwritten_digits_test/1/hsf_0_00066.png': 7, '/home/venkateshiyer/handwritten_digits_test/1/hsf_0_00407.png': 7}


In [16]:
print(accuracy)

0.996078431372549


In [17]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.996078431372549
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       510
           7       0.00      0.00      0.00         0

    accuracy                           1.00       510
   macro avg       0.50      0.50      0.50       510
weighted avg       1.00      1.00      1.00       510



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# For 2 

In [18]:
path = '/home/venkateshiyer/handwritten_digits_test/2/*'
result, accuracy, gt, prediction, wrong_imgs = pred_using_h5_digit(model,path)

In [19]:
print(accuracy)

0.42745098039215684


In [ ]:
# For 3

In [6]:
path = '/home/venkateshiyer/handwritten_digits_test/3/*'
result, accuracy, gt, prediction, wrong_imgs = pred_using_h5_digit(model,path)

2022-12-09 10:10:08.242628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [7]:
print(accuracy)

0.5294117647058824


In [8]:
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           3       1.00      0.53      0.69       510
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.53       510
   macro avg       0.11      0.06      0.08       510
weighted avg       1.00      0.53      0.69       510



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# For 5

In [9]:
path = '/home/venkateshiyer/handwritten_digits_test/5/*'
result, accuracy, gt, prediction, wrong_imgs = pred_using_h5_digit(model,path)

In [10]:
print(accuracy)

0.6568627450980392


In [11]:
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.66      0.79       510
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.66       510
   macro avg       0.14      0.09      0.11       510
weighted avg       1.00      0.66      0.79       510



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
